## SANS2D reduction
This is a notebook to reduce data from SANS2D. 

In [ ]:
import scipp as sc
import scippneutron as scn
from scipp.plotting import plot
from ess.loki.load_files import load_isis, load_rkh_q, load_rkh_wav, load_and_apply_masks
from transform_coordinates import setup_offsets, setup_geometry
from ess.sans import sans

## Loading files

In [ ]:
try:
    import dataconfig # run make_config.py to create this
except:
    print("ERROR: dataconfig.py not find please run `make_config.py`\n")
    !python make_config.py -h # change to `-f path` or run in terminal

path = dataconfig.data_root
direct_beam_file = 'DIRECT_SANS2D_REAR_34327_4m_8mm_16Feb16.dat'

sample_run_number = 63114
sample_transmission_run_number = 63114
background_run_number = 63159
background_transmission_run_number = 63159 
direct_run_number = 63091

mask_1 = f'{path}/MASK_SANS2D_REAR_Edges_16Mar2015.xml'
mask_2 = f'{path}/MASK_SANS2D_FRONT_Edges_16Mar2015.xml'
mask_3 = f'{path}/MASK_SANS2D_BOTH_Extras_24Mar2015.xml'
mask_4 = f'{path}/MASK_SANS2D_REAR_Bottom_3_tubes_16May2014.xml'
mask_5 = f'{path}/MASK_SANS2D_beam_stop_4m_x_100mm_2July2015_medium_beamstop.xml'
mask_6 = f'{path}/MASK_SANS2D_REAR_module2_tube12.xml'
mask_7 = f'{path}/MASK_SANS2D_FRONT_module3_tube14_module5_tube6_tube7.xml'
mask_8 = f'{path}/MASK_SANS2D_FRONT_module2_tube18.xml'
mask_9 = f'{path}/MASK_SANS2D_FRONT_module5_tube20.xml'

idf_filename = f'{path}/SANS2D_Definition_Tubes.xml'

In [ ]:
%%time
direct_beam = load_rkh_wav(filename=f'{path}/{direct_beam_file}')

data_set = sc.Dataset()
#Using only one-forth of the full spectra 245760 (reserved for first detector)
spectrum_size =  245760//4
tof_bins = sc.linspace(dim = 'tof', start = 0, stop = 100000, num = 2, unit=sc.units.us)

data_set['sample'] =  load_isis(filename=f'{path}/SANS2D000{sample_run_number}.nxs', spectrum_size = spectrum_size, tof_bins = tof_bins)
data_set['background'] = load_isis(filename=f'{path}/SANS2D000{background_run_number}.nxs', spectrum_size = spectrum_size, tof_bins = tof_bins)
data_set['direct'] = load_isis(filename=f'{path}/SANS2D000{direct_run_number}.nxs', spectrum_size = spectrum_size, tof_bins = tof_bins)
data_set['sample_trans'] = load_isis(filename=f'{path}/SANS2D000{sample_transmission_run_number}.nxs', spectrum_size = spectrum_size, tof_bins = tof_bins)
data_set['background_trans'] = load_isis(filename=f'{path}/SANS2D000{background_transmission_run_number}.nxs', spectrum_size =  spectrum_size, tof_bins = tof_bins)


## Setting up masks and geometries

In [ ]:
mask_files = [mask_1, mask_2, mask_3, mask_4, mask_5, mask_6, mask_7, mask_8, mask_9]
load_and_apply_masks(idf_filename, mask_files, data_set, spectrum_size)                                                           
apply_tof_mask(data_set)

## Defining parameters for data reduction 

In [ ]:
q_bins = sc.linspace(dim = 'Q', start = 0.0075, stop = 0.5225, num = 104, unit = sc.units.one/sc.units.angstrom)
wavelength_bins = sc.linspace(dim = 'wavelength', start = 2, stop = 10, num = 118, unit=sc.units.angstrom)
wavelength_bands = sc.linspace(dim = 'wavelength', start = 2, stop = 10, num = 9, unit=sc.units.angstrom)
#Min and Max bins for M2 (incident) monitor 
min_bin_mon2 = 85000.0 * sc.units.us
max_bin_mon2 = 98000.0 * sc.units.us
min_bin_mon4 = 85000.0 * sc.units.us
max_bin_mon4 = 98000.0 * sc.units.us
tof_bins_monitors = [min_bin_mon_incident, max_bin_mon_incident, 
                     min_bin_mon_transmission, max_bin_mon_transmission] 
#Solid angle values
pixel_size = 0.00405 * sc.units.m
pixel_length = 0.002033984375 * sc.units.m

#Coordinate transformation
sample_pos_z_offset = 0.053 * sc.units.m
bench_pos_y_offset = 0.001 * sc.units.m
monitor4_pos_z_offset = -6.718 * sc.units.m

#Geometry transformation
x_offset = -0.09288 * sc.units.m
y_offset = 0.08195 * sc.units.m
z_offset = 0.0 * sc.units.m

### Do coordinate transformations

In [ ]:
setup_offsets(data_set, sample_pos_z_offset, bench_pos_y_offset, monitor4_pos_z_offset)

In [ ]:
setup_geometry(data_set, x_offset, y_offset, z_offset)

## Reduction (full spectra)

In [ ]:
%%time
sample_q_reduce = sans.to_q(data=data_set['sample'],
                        transmission=data_set['sample_trans'],
                        direct_beam=direct_beam,
                        direct = data_set['direct'], 
                        masks=data_set['sample'].masks,
                        q_bins = q_bins,
                        tof_bins_monitors = tof_bins_monitors,
                        pixel_size = pixel_size, 
                        pixel_length = pixel_length,
                        wavelength_bins = wavelength_bins)

In [ ]:
%%time
background_q_reduce = sans.to_q(data=data_set['background'],
                            transmission=data_set['background_trans'],
                            direct_beam=direct_beam,
                            direct = data_set['direct'], # note: background_trans
                            masks=data['background'].masks,
                            q_bins = q_bins,
                            tof_bins_monitors = tof_bins_monitors,
                            pixel_size = pixel_size, 
                            pixel_length = pixel_length,
                            wavelength_bins = wavelength_bins)

In [ ]:
reduced = sans.normalize_and_subtract(sample_q_reduce, background_q_reduce)

## Reduction by wavelength

In [ ]:
%%time
sample_q_lambda = sans.to_q(data=data_set['sample'],
                            transmission=data_set['sample_trans'],
                            direct_beam=direct_beam,
                            direct = data_set['direct'], # note: background_trans
                            masks=data_set['sample'].masks,
                            q_bins = q_bins,
                            tof_bins_monitors = tof_bins_monitors,
                            pixel_size = pixel_size, 
                            pixel_length = pixel_length,
                            wavelength_bins = wavelength_bins,
                            wavelength_bands = wavelength_bands)

In [ ]:
%%time
background_q_lambda = sans.to_q(data=data_set['background'],
                                transmission=data_set['background_trans'],
                                direct_beam=direct_beam,
                                direct = data_set['direct'], # note: same as transmission
                                masks=data_set['background'].masks,
                                q_bins = q_bins,
                                tof_bins_monitors = tof_bins_monitors,
                                pixel_size = pixel_size, 
                                pixel_length = pixel_length,
                                wavelength_bins = wavelength_bins,
                                wavelength_bands = wavelength_bands)

In [ ]:
sample_q_reduce_wav = sc.sum(sample_q_lambda, 'wavelength')
background_q_reduce_wav = sc.sum(background_q_lambda, 'wavelength')

In [ ]:
reduced_by_wavelength = sans.normalize_and_subtract(sample_q_reduce_wav, background_q_reduce_wav)

## Comparison reduced vs reduced_by_wavelength

In [ ]:
plot({'reduced': reduced, 'reduced_by_wavelength':reduced_by_wavelength})

In [ ]:
reduced_lambda = sans.normalize_and_subtract(sample_q_lambda, background_q_lambda)
sc.plot(sc.collapse(reduced_lambda, keep='Q'))